*ENUNCIADO*


Considere um título com valor de Face igual a 1000 (reais) e 40 (reais) de coupon

O título vence em cinco anos, paga coupons anuais e possui rating BB.  

Estime o VaR de Crédito pela distribuição empírica e também usando a distribuição normal. 

(utilize um ano como horizonte temporal)

Dados do problema:

In [2]:
import pandas
matriz_transicao = pandas.read_csv("Q1_matriz_transicao.tsv", sep="\t").set_index("Rating")
matriz_transicao

,AAA,AA,A,BBB,BB,B,CCC,Default
Rating,,,,,,,,
AAA,90.81,8.33,0.68,0.06,0.12,0.00,0.00,0.00
AA,0.70,90.65,7.79,0.64,0.06,0.14,0.02,0.00
A,0.09,2.27,91.05,5.52,0.74,0.26,0.01,0.06
BBB,0.02,0.33,5.95,86.93,5.30,1.17,0.12,0.18
BB,0.03,0.14,0.67,7.73,80.53,8.84,1.00,1.06
B,0.00,0.11,0.24,0.43,6.48,83.46,4.07,5.20
CCC,0.22,0.00,0.22,1.30,2.38,11.24,64.86,19.79


In [3]:
forward_zero_curves = pandas.read_csv("Q1_forward_zero_curves.tsv", sep="\t").set_index("Rating")
forward_zero_curves

,Y1,Y2,Y3,Y4
Rating,,,,
AAA,3.60,4.17,4.73,5.12
AA,3.65,4.22,4.78,5.17
A,3.72,4.32,4.93,5.32
BBB,4.10,4.67,5.25,5.63
BB,5.55,6.02,6.78,7.27
B,6.05,7.02,8.03,8.52
CCC,15.05,15.02,14.03,13.52
Default,NaN,NaN,NaN,NaN


A precificação do título após um ano é dada pela seguinte equação:

$$P = 40 + \frac{40}{(1+r_1)^1} + \frac{40}{(1+r_2)^2} + \frac{40}{(1+r_3)^3} + \frac{1040}{(1+r_4)^4}$$

Agora vamos computar os fluxos de caixa para cada taxa e precificar o título para cada cenário:


In [4]:
cashflow = forward_zero_curves.copy()
cashflow["CF0"] = 40.00
cashflow["CF1"] = round(40 / (1 + forward_zero_curves["Y1"] / 100)**1, 2)
cashflow["CF2"] = round(40 / (1 + forward_zero_curves["Y2"] / 100)**2, 2)
cashflow["CF3"] = round(40 / (1 + forward_zero_curves["Y3"] / 100)**3, 2)
cashflow["CF4"] = round(1040 / (1 + forward_zero_curves["Y4"] / 100)**4, 2)
cashflow["P"] = cashflow[["CF0", "CF1", "CF2", "CF3", "CF4"]].sum(axis=1)
cashflow.iloc[-1] = 0
cashflow

,Y1,Y2,Y3,Y4,CF0,CF1,CF2,CF3,CF4,P
Rating,,,,,,,,,,
AAA,3.60,4.17,4.73,5.12,40.0,38.61,36.86,34.82,851.71,1002.00
AA,3.65,4.22,4.78,5.17,40.0,38.59,36.83,34.77,850.09,1000.28
A,3.72,4.32,4.93,5.32,40.0,38.57,36.76,34.62,845.26,995.21
BBB,4.10,4.67,5.25,5.63,40.0,38.42,36.51,34.31,835.38,984.62
BB,5.55,6.02,6.78,7.27,40.0,37.90,35.59,32.85,785.45,931.79
B,6.05,7.02,8.03,8.52,40.0,37.72,34.92,31.73,749.88,894.25
CCC,15.05,15.02,14.03,13.52,40.0,34.77,30.24,26.98,626.24,758.23
Default,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00


Como é um título BB, transpomos a linha correspondente da matriz de transição numa nova coluna, levando as probabilidades de mudança do rating do título:

In [5]:
cashflow["p"] = matriz_transicao.iloc[4]
cashflow

,Y1,Y2,Y3,Y4,CF0,CF1,CF2,CF3,CF4,P,p
Rating,,,,,,,,,,,
AAA,3.60,4.17,4.73,5.12,40.0,38.61,36.86,34.82,851.71,1002.00,0.03
AA,3.65,4.22,4.78,5.17,40.0,38.59,36.83,34.77,850.09,1000.28,0.14
A,3.72,4.32,4.93,5.32,40.0,38.57,36.76,34.62,845.26,995.21,0.67
BBB,4.10,4.67,5.25,5.63,40.0,38.42,36.51,34.31,835.38,984.62,7.73
BB,5.55,6.02,6.78,7.27,40.0,37.90,35.59,32.85,785.45,931.79,80.53
B,6.05,7.02,8.03,8.52,40.0,37.72,34.92,31.73,749.88,894.25,8.84
CCC,15.05,15.02,14.03,13.52,40.0,34.77,30.24,26.98,626.24,758.23,1.00
Default,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,1.06


Para precificar o título daqui a um ano, multiplica-se os preços com as probabilidades e achar o preço esperado em cada cenário:

In [6]:
cashflow["P*p"] = round(cashflow["P"] * cashflow["p"] / 100, 2)
cashflow

,Y1,Y2,Y3,Y4,CF0,CF1,CF2,CF3,CF4,P,p,P*p
Rating,,,,,,,,,,,,
AAA,3.60,4.17,4.73,5.12,40.0,38.61,36.86,34.82,851.71,1002.00,0.03,0.30
AA,3.65,4.22,4.78,5.17,40.0,38.59,36.83,34.77,850.09,1000.28,0.14,1.40
A,3.72,4.32,4.93,5.32,40.0,38.57,36.76,34.62,845.26,995.21,0.67,6.67
BBB,4.10,4.67,5.25,5.63,40.0,38.42,36.51,34.31,835.38,984.62,7.73,76.11
BB,5.55,6.02,6.78,7.27,40.0,37.90,35.59,32.85,785.45,931.79,80.53,750.37
B,6.05,7.02,8.03,8.52,40.0,37.72,34.92,31.73,749.88,894.25,8.84,79.05
CCC,15.05,15.02,14.03,13.52,40.0,34.77,30.24,26.98,626.24,758.23,1.00,7.58
Default,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,1.06,0.00


Somando esta coluna encontra-se o preço esperado:

In [7]:
expected_price = cashflow["P*p"].sum()
expected_price

921.48

Com isso, e supondo distribuição normal de preços, vamos calcular primeiro a variância:

In [8]:
cashflow["Variance"] = round(cashflow["p"] / 100 * (cashflow["P"] - expected_price)**2, 2)
cashflow

,Y1,Y2,Y3,Y4,CF0,CF1,CF2,CF3,CF4,P,p,P*p,Variance
Rating,,,,,,,,,,,,,
AAA,3.60,4.17,4.73,5.12,40.0,38.61,36.86,34.82,851.71,1002.00,0.03,0.30,1.95
AA,3.65,4.22,4.78,5.17,40.0,38.59,36.83,34.77,850.09,1000.28,0.14,1.40,8.69
A,3.72,4.32,4.93,5.32,40.0,38.57,36.76,34.62,845.26,995.21,0.67,6.67,36.42
BBB,4.10,4.67,5.25,5.63,40.0,38.42,36.51,34.31,835.38,984.62,7.73,76.11,308.17
BB,5.55,6.02,6.78,7.27,40.0,37.90,35.59,32.85,785.45,931.79,80.53,750.37,85.60
B,6.05,7.02,8.03,8.52,40.0,37.72,34.92,31.73,749.88,894.25,8.84,79.05,65.55
CCC,15.05,15.02,14.03,13.52,40.0,34.77,30.24,26.98,626.24,758.23,1.00,7.58,266.51
Default,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,1.06,0.00,9000.73


E com isso podemos calcular o desvio padrão:

In [13]:
import math
stdev = round(math.sqrt(cashflow["Variance"].sum()), 2)
stdev

98.86

E o VaR de crédito no IC 99%:

In [14]:
from scipy.stats import norm
var = round(norm.ppf(0.01) * stdev, 2)
var

-229.98

Pela distribuição empírica, calculamos a probabilidade acumulada:

In [11]:
cashflow["Cumulative"] = cashflow["p"].loc[::-1].cumsum().loc[::-1]
cashflow

,Y1,Y2,Y3,Y4,CF0,CF1,CF2,CF3,CF4,P,p,P*p,Variance,Cumulative
Rating,,,,,,,,,,,,,,
AAA,3.60,4.17,4.73,5.12,40.0,38.61,36.86,34.82,851.71,1002.00,0.03,0.30,1.95,100.00
AA,3.65,4.22,4.78,5.17,40.0,38.59,36.83,34.77,850.09,1000.28,0.14,1.40,8.69,99.97
A,3.72,4.32,4.93,5.32,40.0,38.57,36.76,34.62,845.26,995.21,0.67,6.67,36.42,99.83
BBB,4.10,4.67,5.25,5.63,40.0,38.42,36.51,34.31,835.38,984.62,7.73,76.11,308.17,99.16
BB,5.55,6.02,6.78,7.27,40.0,37.90,35.59,32.85,785.45,931.79,80.53,750.37,85.60,91.43
B,6.05,7.02,8.03,8.52,40.0,37.72,34.92,31.73,749.88,894.25,8.84,79.05,65.55,10.90
CCC,15.05,15.02,14.03,13.52,40.0,34.77,30.24,26.98,626.24,758.23,1.00,7.58,266.51,2.06
Default,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,1.06,0.00,9000.73,1.06


Daí pegamos o percentil mais próximo (Default) e o VaR será:

In [12]:
var = cashflow["P"].loc["Default"] - expected_price
var

-921.48